In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

# 설정C:\Users\Developer\TCLab\csv
csv_folder = Path("C:\\Users\\Developer\\TCLab\\csv")

csv_files = sorted(csv_folder.glob("mpc_episode_*_data.csv"))
# C:\Users\Developer\TCLab\csv\data_back4
E1, E2 = 1.0, 1.0  # 가중치 설정

all_observations = []
all_actions = []
all_next_observations = []
all_rewards = []
all_dones = []

for file in csv_files:
    df = pd.read_csv(file)
    for i in range(len(df) - 1):
        curr = df.iloc[i]
        next_ = df.iloc[i + 1]
        
        state = [curr["T1"], curr["T2"], curr["TSP1"], curr["TSP2"]]
        action = [curr["Q1"], curr["Q2"]]
        next_state = [next_["T1"], next_["T2"], next_["TSP1"], next_["TSP2"]]
        
        # L2 norm 기반 reward
        error_vec = np.array([curr["TSP1"] - curr["T1"], curr["TSP2"] - curr["T2"]])
        reward = -np.linalg.norm(error_vec)
        
        done = (i == len(df) - 2)

        all_observations.append(state)
        all_actions.append(action)
        all_next_observations.append(next_state)
        all_rewards.append(reward)
        all_dones.append(done)

# numpy 배열로 변환
dataset = {
    "observations": np.array(all_observations, dtype=np.float32),
    "actions": np.array(all_actions, dtype=np.float32),
    "next_observations": np.array(all_next_observations, dtype=np.float32),
    "rewards": np.array(all_rewards, dtype=np.float32),
    "terminals": np.array(all_dones, dtype=bool),
}
print("Reward 평균:", np.mean(all_rewards))
print("Reward 최소:", np.min(all_rewards))
print("Reward 최대:", np.max(all_rewards))

# 저장
output_path_l2 = csv_folder.parent / "mpc_dataset.npz"
np.savez(output_path_l2, **dataset)



Reward 평균: -9.310084092646807
Reward 최소: -47.412083902735176
Reward 최대: -0.00999999999999801


reward = -np.linalg.norm(error_vec) * 0.1


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path

# 설정C:\Users\Developer\TCLab\csv
csv_folder = Path("C:\\Users\\Developer\\TCLab\\csv")
csv_files = sorted(csv_folder.glob("mpc_episode_*_data.csv"))

E1, E2 = 1.0, 1.0  # 가중치 설정

all_observations = []
all_actions = []
all_next_observations = []
all_rewards = []
all_dones = []

for file in csv_files:
    df = pd.read_csv(file)
    for i in range(len(df) - 1):
        curr = df.iloc[i]
        next_ = df.iloc[i + 1]
        
        state = [curr["T1"], curr["T2"], curr["TSP1"], curr["TSP2"]]
        action = [curr["Q1"], curr["Q2"]]
        next_state = [next_["T1"], next_["T2"], next_["TSP1"], next_["TSP2"]]
        
        # L2 norm 기반 reward
        error_vec = np.array([curr["TSP1"] - curr["T1"], curr["TSP2"] - curr["T2"]])
        reward = -np.linalg.norm(error_vec) * 0.1
        
        done = (i == len(df) - 2)

        all_observations.append(state)
        all_actions.append(action)
        all_next_observations.append(next_state)
        all_rewards.append(reward)
        all_dones.append(done)

# numpy 배열로 변환
dataset = {
    "observations": np.array(all_observations, dtype=np.float32),
    "actions": np.array(all_actions, dtype=np.float32),
    "next_observations": np.array(all_next_observations, dtype=np.float32),
    "rewards": np.array(all_rewards, dtype=np.float32),
    "terminals": np.array(all_dones, dtype=bool),
}
print("Reward 평균:", np.mean(all_rewards))
print("Reward 최소:", np.min(all_rewards))
print("Reward 최대:", np.max(all_rewards))

acts = dataset['actions']

print("[Debug] Action range in dataset:", acts.min(), "~", acts.max())
print("[Debug] Action mean:", acts.mean(axis=0), "std:", acts.std(axis=0))

# 저장
# output_path_l2 = csv_folder.parent / "mpc_dataset1.npz"
# np.savez(output_path_l2, **dataset)



Reward 평균: -0.931008409264681
Reward 최소: -4.741208390273518
Reward 최대: -0.0009999999999998012
[Debug] Action range in dataset: 0.0 ~ 100.0
[Debug] Action mean: [30.623505 40.10309 ] std: [42.8137   44.337543]


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

# 두 개의 csv 디렉토리 경로
csv_folders = [
    Path("C:/Users/Developer/TCLab/csv"),
    Path("C:/Users/Developer/TCLab/csv/data_back4")
]

E1, E2 = 1.0, 1.0  # 가중치 설정

all_observations = []
all_actions = []
all_next_observations = []
all_rewards = []
all_dones = []

# 모든 폴더에서 CSV 병합
for folder in csv_folders:
    csv_files = sorted(folder.glob("mpc_episode_*_data.csv"))
    for file in csv_files:
        df = pd.read_csv(file)
        for i in range(len(df) - 1):
            curr = df.iloc[i]
            next_ = df.iloc[i + 1]
            
            state = [curr["T1"], curr["T2"], curr["TSP1"], curr["TSP2"]]
            action = [curr["Q1"], curr["Q2"]]
            next_state = [next_["T1"], next_["T2"], next_["TSP1"], next_["TSP2"]]
            
            # L2 norm 기반 reward
            error_vec = np.array([curr["TSP1"] - curr["T1"], curr["TSP2"] - curr["T2"]])
            reward = -np.linalg.norm(error_vec)
            reward += 10.0  # shift: 음수를 양수로
            reward *= 0.1   # scale: 보상 범위 조정

          #  reward += 10.0    # 적절한 offset
          #  reward *=     
            
            done = (i == len(df) - 2)

            all_observations.append(state)
            all_actions.append(action)
            all_next_observations.append(next_state)
            all_rewards.append(reward)
            all_dones.append(done)

# numpy 배열로 변환
dataset = {
    "observations": np.array(all_observations, dtype=np.float32),
    "actions": np.array(all_actions, dtype=np.float32),
    "next_observations": np.array(all_next_observations, dtype=np.float32),
    "rewards": np.array(all_rewards, dtype=np.float32),
    "terminals": np.array(all_dones, dtype=bool),
}

# 리워드 통계 출력
print("Reward 평균:", np.mean(all_rewards))
print("Reward 최소:", np.min(all_rewards))
print("Reward 최대:", np.max(all_rewards))

acts = dataset['actions']

print("[Debug] Action range in dataset:", acts.min(), "~", acts.max())
print("[Debug] Action mean:", acts.mean(axis=0), "std:", acts.std(axis=0))

rewards = dataset['rewards']
print("[Debug] rewards range in dataset:", rewards.min(), "~", rewards.max())
print("[Debug] rewards mean:", rewards.mean(axis=0), "std:", rewards.std(axis=0))

observations = dataset['observations']
print("[Debug] observations range in dataset:", observations.min(), "~", observations.max())
print("[Debug] observations mean:", observations.mean(axis=0), "std:", observations.std(axis=0))
# 저장 경로 (부모 디렉토리에 저장)
output_path = csv_folders[0].parent / "mpc_dataset_merged.npz"
np.savez(output_path, **dataset)
print(f"✅ 저장 완료: {output_path}")


Reward 평균: 0.20997810061982278
Reward 최소: -3.741208390273518
Reward 최대: 0.9990000000000002
[Debug] Action range in dataset: 0.0 ~ 100.0
[Debug] Action mean: [26.522545 40.997356] std: [40.481014 43.941162]
[Debug] rewards range in dataset: -3.7412083 ~ 0.999
[Debug] rewards mean: 0.20997809 std: 0.86896694
[Debug] observations range in dataset: 25.22 ~ 68.24
[Debug] observations mean: [44.730286 43.0406   44.32769  43.805374] std: [10.942937  9.281088 11.969531 10.855006]
